# Avaliação de Medidas (In)Coerência do conjunto de dados Cohebert Manual usando BERT Pré-treinado

Este notebook, realiza testes de medidas de (in)coerência entre pares de documentos do conjunto de dados Cohebert utilizando BERT Pré-treinado. 

A medida de um documento(D) é realizada pela média da medida dos pares de palavras. Todo o documento é submetido ao BERT e os embeddings da concatenação das 4 últimas camadas é recuperado. Cada documento é analisado pelas medidas **Ccos**, **Ceuc** e **Cman**. Estas medidas de documento utilizam as medidas de palavras adjancentes.Cada par de palavras do documento é analisado utilizando os embedding da palavra
(**wi**) e os embeddings da palavra(**wj**) no documento.

As seguintes medidas foram calculadas entre os embeddings das sentenças **wi** e **wj**:
- **Scos(wi,wj)** - Similaridade do cosseno entre a média dos embeddings Si e Sj.
- **Seuc(wi,wj)** - Distância euclidiana - usando a média dos embeddings Si e Sj das camadas especificadas
- **Sman(wi,wj)** - Distância de manhattan - usando a média dos embeddings wi e wj das camadas especificadas

As medidas são avalidas considerando a concatenação das 4 últimas do modelo BERT.

Características dos testes:
 - A avaliação das medidas são realizadas utilizando a implementação BERT da biblioteca Huggingface e os MCL BERT **Pré-treinado** no formato cased:
  - BERTimbau de tamanho Large 
  
 - Não é realizado ajuste fino do modelo.

Utiliza os arquivos para gerar as medidas:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`
- `comparacao_palavra_pX_kY.zip`
- `comparacao_contexto_pX_kY.zip`

Nos nomes dos arquivos, `X` é o número de documentos perturbados e `Y` o valor de top `K` predições. 


----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas
import time
import datetime

#marca o tempo de início do processamento.
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={'help': 'max seq len'},
    )    
    pretrained_model_name_or_path: str = field(
        default='neuralmind/bert-base-portuguese-cased',
        metadata={'help': 'nome do modelo pré-treinado do BERT.'},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={'help': 'define se o texto do modelo deve ser todo em minúsculo.'},
    )  
    output_attentions: bool = field(
        default=False,
        metadata={'help': 'habilita se o modelo retorna os pesos de atenção.'},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={'help': 'habilita gerar as camadas ocultas do modelo.'},
    )
    use_wandb : bool = field(
        default=True,
        metadata={'help': 'habilita o uso do wandb.'},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={'help': 'habilita o salvamento do resultado da avaliação.'},
    )     
    salvar_medicao : bool = field(
        default=False,
        metadata={'help': 'habilita o salvamento da medicao.'},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )
    estrategia_medida: int = field(
        default=0, # 0 - MEAN estratégia média / 1 - MAX  estratégia maior
        metadata={'help': 'Estratégia de cálculo da médida dos embeddings.'},
    )
    equacao_medida: int = field(
        default=0, # 0 - ADJACENTE / 1 - COMBINAÇÃO TODAS / 2 - CONTEXTO
        metadata={'help': 'Equação de cálculo da coerência.'},
    )
    filtro_palavra: int = field(
        default=0, # 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
        metadata={'help': 'Define o filtro de palavras das sentenças para gerar os embeddings.'},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.4  Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory.
import sys

# Retorna true ou false se estiver no Google Colaboratory.
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

2022-07-12 00:22:43,360 : INFO : NumExpr defaulting to 2 threads.
2022-07-12 00:22:44,280 : INFO : Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

2022-07-12 00:22:45,913 : INFO : Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

2022-07-12 00:22:45,930 : INFO : Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

2022-07-12 00:22:45,933 : INFO : Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
2022-07-12 00:22:45,936 : INFO : e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.9 Instalação do BERT

Instala a interface pytorch para o BERT by Hugging Face. 

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

In [ ]:
!pip install -U transformers==4.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 Parametrização

## Gerais

In [ ]:
# Nome base das saidas do projeto
NOME_BASE_SAIDA = "MedidaCoerenciaCohebertManual_v1"

# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1

# Equação de coerência (0 - Palavras adjacentes / 1 - Combinação de todas as palavras / 2 - Palavra e contexto) 
EQUACAO_MEDIDA_STR = ["ADJACENTES","COMBINAÇÃO","CONTEXTO"]
EQUACAO_MEDIDA = [0, 1, 2]

# Estratégias a serem avaliadas (0 - Mean / 1 - Max) para as palavras formadas por mais de um token do BERT
ESTRATEGIA_MEDIDA_STR = ["MEAN", "MAX"]
ESTRATEGIA_MEDIDA = [0, 1]

# Filtro de palavras das sentenças[0,1,2] 'TAP,SSW,SVS'
FILTRO_PALAVRAS_STR = ["TODAS_AS_PALAVRAS", 
                       "SEM_STOPWORDS",                        
                       "SOMENTE_VERBOS_SUBSTANTIVOS"]
FILTRO_PALAVRAS = [0, 1, 2]

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512,    
    # pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    versao_modelo_spacy = "3.2.0",
    do_lower_case = False,  # default True
    output_attentions = False,  # default False
    output_hidden_states = True, # default False
    use_wandb = True,    
    salvar_medicao = True, #Salva o resultado da medição
    salvar_avaliacao = True, # Salva o resultado da avaliação das medições
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.    
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara. 
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
    estrategia_medida = 0, # Atributo usado para os logs do wandb. 0 - MEAN estratégia média / 1 - MAX  estratégia maior
    equacao_medida = 0, # Atributo usado para os logs do wandb. 0 - Palavras adjacentes / 1 - Todas as palavras / 2 - Palavra e contexto
    filtro_palavra = 0 # # Atributo usado para os logs do wandb. 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHEBERT_MANUAL"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):  
      # Cria o diretório
      os.makedirs(dirbase)    
      logging.info("Diretório criado: {}.".format(dirbase))
  else:    
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

2022-07-12 00:23:19,494 : INFO : Diretório já existe: /content/COHEBERT_MANUAL.


Diretório para conter as os resultados das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedidacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedidacao()

2022-07-12 00:23:19,535 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra.


Diretório para conter os arquivos da avaliação

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioAvaliacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Avaliacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioAvaliacao()

2022-07-12 00:23:19,593 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra/Avaliacao.


Diretório para conter os arquivos das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedicao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedicao()

2022-07-12 00:23:19,626 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra/Medicao.


# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivoTar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivoTar.extractall(DIRETORIO_COHEBERT)    
    arquivoTar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

2022-07-12 00:23:25,791 : INFO : spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

2022-07-12 00:23:25,822 : INFO : Quantidade de stopwords: 416.


{'estive', 'entre', 'número', 'vos', 'possível', 'quinto', 'cada', 'menor', 'das', 'vossas', 'mais', 'qual', 'estado', 'sexta', 'pois', 'estiveram', 'catorze', 'sem', 'ali', 'ela', 'vossos', 'os', 'suas', 'fazer', 'mas', 'contra', 'uns', 'tentar', 'este', 'vão', 'demais', 'outras', 'todas', 'uma', 'estiveste', 'novas', 'sétimo', 'tens', 'além', 'maior', 'tiveram', 'pelos', 'des', 'cinco', 'dessa', 'talvez', 'mesmo', 'dos', 'maiorias', 'depois', 'apontar', 'pode', 'quer', 'baixo', 'estivemos', 'logo', 'ao', 'duas', 'vezes', 'ambos', 'geral', 'outra', 'umas', 'próprio', 'momento', 'quais', 'custa', 'não', 'boa', 'fará', 'podem', 'quinta', 'seis', 'quarto', 'a', 'vários', 'aquilo', 'irá', 'meses', 'tente', 'sou', 'nova', 'desta', 'tendes', 'somente', 'és', 'fomos', 'mal', 'põe', 'fazes', 'último', 'nível', 'somos', 'tais', 'faz', 'vosso', 'portanto', 'naquela', 'tarde', 'tenho', 'conselho', 'isso', 'grande', 'tempo', 'dezasseis', 'me', 'ainda', 'enquanto', 'aqui', 'naquele', 'toda', 'inic

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    listatokens.append(token.text)
    listapos.append(token.pos_)
    
  return listatokens, listapos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, classe_saliente=["NOUN"]):
  
  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especificado
    if pos[i] in classe_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):
  
  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documento_limpo

# 4 Funções auxiliares

## getNomeModeloBERT

In [ ]:
def getNomeModeloBERT(model_args):
    '''    
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"
    
    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"
        
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
            
    return MODELO_BERT

## getTamanhoBERT

In [ ]:
def getTamanhoBERT(model_args):
    '''    
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''
    
    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"
    
    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"
        
    return TAMANHO_BERT  

## concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  listaConcat = []

  for x in lista:
      listaConcat = listaConcat + x[pos]
  
  return listaConcat

## encontrarIndiceSubLista 

Retorna os índices de início e fim da sublista na lista

In [ ]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):

    """
      Localiza os índices de início e fim de uma sublista em uma lista.
    
      Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.
    """    
    # https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

# 5 Comparar documentos





## 5.1 Carregamento dos arquivos de dados originais 

#### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

### 5.1.2 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"
 
  logging.info("Terminei a cópia.")

2022-07-12 00:23:26,576 : INFO : Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

2022-07-12 00:23:26,928 : INFO : Terminei a descompactação.


### 5.1.3 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-07-12 00:23:26,961 : INFO : TERMINADO ORIGINAIS: 40.
2022-07-12 00:23:26,963 : INFO : TERMINADO ORIGINAIS POS: 40.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento,classe
3,4,['Como desenfileirar elementos em uma pilha?'],Como desenfileirar elementos em uma pilha?,0
36,37,['Em uma pilha a operação de empilhar ocorre e...,Em uma pilha a operação de empilhar ocorre em ...,1
30,31,['O que é uma fila e como enfileirar e desenfi...,O que é uma fila e como enfileirar e desenfile...,1
8,9,['Como empilhar elementos em uma estrutura de ...,Como empilhar elementos em uma estrutura de da...,1
21,22,['O que é uma pilha e como desenfileirar um el...,O que é uma pilha e como desenfileirar um elem...,0


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
29,30,"[[['O', 'que', 'é', 'uma', 'fila', 'e', 'como'..."
22,23,"[[['O', 'que', 'é', 'uma', 'pilha', 'e', 'como..."
31,32,"[[['O', 'que', 'é', 'uma', 'pilha', 'e', 'como..."
15,16,"[[['Como', 'desempilhar', 'elementos', 'em', '..."
7,8,"[[['Como', 'empilhar', 'e', 'desempilhar', 'el..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

2022-07-12 00:23:27,054 : INFO : TERMINADO CORREÇÃO ORIGINAIS: 40.
2022-07-12 00:23:27,056 : INFO : TERMINADO CORREÇÃO ORIGINAIS POS: 40.


#### Criando dados indexados originais

In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_originais_indexado = lista_documentos_originais.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_originais_indexado.set_index(["id"], inplace=True)
lista_documentos_originais_indexado.head()

,sentencas,documento,classe
id,,,
1,[Como enfileirar elementos em uma fila?],Como enfileirar elementos em uma fila?,1
2,[Como enfileirar elementos em uma pilha?],Como enfileirar elementos em uma pilha?,0
3,[Como desenfileirar elementos em uma fila?],Como desenfileirar elementos em uma fila?,1
4,[Como desenfileirar elementos em uma pilha?],Como desenfileirar elementos em uma pilha?,0
5,[Como empilhar elementos em uma pilha?],Como empilhar elementos em uma pilha?,1


In [ ]:
# Faz uma cópia da lista não indexada
lista_documentos_originais_pos_indexado = lista_documentos_originais_pos.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_documentos_originais_pos_indexado.set_index(["id"], inplace=True)
lista_documentos_originais_pos_indexado.head()

,pos_documento
id,
1,"[[[Como, enfileirar, elementos, em, uma, fila,..."
2,"[[[Como, enfileirar, elementos, em, uma, pilha..."
3,"[[[Como, desenfileirar, elementos, em, uma, fi..."
4,"[[[Como, desenfileirar, elementos, em, uma, pi..."
5,"[[[Como, empilhar, elementos, em, uma, pilha, ..."


### 5.1.4 Gerando pares de documentos originais e perturbados / Documento Original(1) e Documento Perturbado(0)



In [ ]:
lista_documentos_agrupados = []

for i, linha in lista_documentos_originais.iterrows():
  #print("classe:", linha['classe'])
  if i % 2 == 0:      
      #print("linha_documento:",linha_documento)
      # Recupera o id do documento
      id_documento_original = linha["id"]
      #print("id_documento_original:",id_documento_original)  
      
      # Carrega a lista das sentenças do documento
      lista_sentencas_original = linha["sentencas"]
      #print("\lista_sentencas_original:",lista_sentencas_original)
      #print("len(lista_sentencas_original):",len(lista_sentencas_original)) 

      # Carrega o documento original
      documento_original = linha["documento"]
      #print("\documento_original:",documento_original)

      # Recupera a POS do documento original
      tokens_original = []
      tokens_original_pos = []
      reg_originais_pos = lista_documentos_originais_pos_indexado.loc[id_documento_original] 
      #print("reg_original_pos:",reg_originais_pos)
      pos_documento_original = reg_originais_pos['pos_documento']
      for i, linha2 in enumerate(pos_documento_original):
        tokens_original.append(linha2[0])
        tokens_original_pos.append(linha2[1])
        
  else:      
      # Id do documento perturbado
      id_perturbado = linha["id"]
      #print("id_perturbado:", id_perturbado)

      # Recupera a sentença do documento perturbado
      lista_sentencas_perturbado = linha["sentencas"]
      #print("\lista_sentencas_perturbado:",lista_sentencas_perturbado)
      #print("len(lista_sentencas_perturbado):",len(lista_sentencas_perturbado)) 

      # Carrega o documento perturbado
      documento_perturbado = linha["documento"]
      #print("\documento_perturbado:",documento_perturbado)
      
      # Recupera a POS do documento perturbado
      tokens_perturbado = []
      tokens_perturbado_pos = []
      reg_perturbado_pos = lista_documentos_originais_pos_indexado.loc[id_perturbado]
      pos_documento_perturbado = reg_perturbado_pos['pos_documento']
      for i, linha2 in enumerate(pos_documento_perturbado):
        tokens_perturbado.append(linha2[0])
        tokens_perturbado_pos.append(linha2[1])

      # Guarda o agrupamento de original e perturbado       
      lista_documentos_agrupados.append([id_documento_original, 
                                         lista_sentencas_original, 
                                         documento_original, 
                                         tokens_original, 
                                         tokens_original_pos,  
                                         id_perturbado, 
                                         lista_sentencas_perturbado, 
                                         documento_perturbado, 
                                         tokens_perturbado, 
                                         tokens_perturbado_pos])

#### Converte a lista em um dataframe

Atributos do dataframe:
0. 'id_original' - Nome do arquivo original
1. 'sentencas_originais' - Lista das sentenças do documento original
2. 'documento_original' - Documento original
3. 'tokens_original' - Tokens do documento original
4. 'pos_original' - Postagging do documento original
5. 'id_perturbado' - Nome do arquivo perturbado
6. 'sentencas_perturbadas' - Lista das sentenças do documento perturbado
7. 'documento_perturbado' - Documento perturbado
8. 'tokens_perturbado' - Tokens do documento perturbado
9. 'pos_perturbado' - Postagging do documento perturbado

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Converte a lista em um dataframe.
lista_documentos_agrupados = pd.DataFrame.from_records(lista_documentos_agrupados, 
                                                         columns=['id_original',
                                                                  'sentencas_original',
                                                                  'documento_original',
                                                                  'tokens_original',
                                                                  'pos_original',
                                                                  'id_perturbado',
                                                                  'sentencas_perturbado',
                                                                  'documento_perturbado',
                                                                  'tokens_perturbado', 
                                                                  'pos_perturbado'])

# Número de linhas carregadas do arquivo.
logging.info('Total de registros              : {}'.format(len(lista_documentos_agrupados)))

2022-07-12 00:23:27,188 : INFO : Total de registros              : 20


In [ ]:
lista_documentos_agrupados.sample(5)

,id_original,sentencas_original,documento_original,tokens_original,pos_original,id_perturbado,sentencas_perturbado,documento_perturbado,tokens_perturbado,pos_perturbado
1,3,[Como desenfileirar elementos em uma fila?],Como desenfileirar elementos em uma fila?,"[[Como, desenfileirar, elementos, em, uma, fil...","[[SCONJ, VERB, NOUN, ADP, DET, NOUN, PUNCT]]",4,[Como desenfileirar elementos em uma pilha?],Como desenfileirar elementos em uma pilha?,"[[Como, desenfileirar, elementos, em, uma, pil...","[[SCONJ, VERB, NOUN, ADP, DET, NOUN, PUNCT]]"
5,11,[Como empilhar e desempilhar elementos em uma ...,Como empilhar e desempilhar elementos em uma e...,"[[Como, empilhar, e, desempilhar, elementos, e...","[[SCONJ, VERB, CCONJ, VERB, NOUN, ADP, DET, NO...",12,[Como empilhar e desempilhar elementos em uma ...,Como empilhar e desempilhar elementos em uma e...,"[[Como, empilhar, e, desempilhar, elementos, e...","[[SCONJ, VERB, CCONJ, VERB, NOUN, ADP, DET, NO..."
7,15,[Como desempilhar elementos em uma estrutura d...,Como desempilhar elementos em uma estrutura de...,"[[Como, desempilhar, elementos, em, uma, estru...","[[ADP, NOUN, NOUN, ADP, DET, NOUN, ADP, NOUN, ...",16,[Como desempilhar elementos em uma estrutura d...,Como desempilhar elementos em uma estrutura de...,"[[Como, desempilhar, elementos, em, uma, estru...","[[ADP, NOUN, NOUN, ADP, DET, NOUN, ADP, NOUN, ..."
17,35,[Como são implementadas as operações de enfile...,Como são implementadas as operações de enfilei...,"[[Como, são, implementadas, as, operações, de,...","[[ADV, AUX, VERB, DET, NOUN, SCONJ, VERB, CCON...",36,[Como são implementadas as operações de enfile...,Como são implementadas as operações de enfilei...,"[[Como, são, implementadas, as, operações, de,...","[[ADV, AUX, VERB, DET, NOUN, SCONJ, VERB, CCON..."
12,25,[O que é uma fila e como enfileirar um element...,O que é uma fila e como enfileirar um elemento...,"[[O, que, é, uma, fila, e, como, enfileirar, u...","[[PRON, PRON, AUX, DET, NOUN, CCONJ, ADV, VERB...",26,[O que é uma pilha e como enfileirar um elemen...,O que é uma pilha e como enfileirar um element...,"[[O, que, é, uma, pilha, e, como, enfileirar, ...","[[PRON, PRON, AUX, DET, NOUN, CCONJ, ADV, VERB..."


## 5.2 Carregamento do arquivo de dado comparação entre palavras

### 5.2.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_COMPARACAO_PALAVRA = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO = "comparacao_palavra_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.2.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

2022-07-12 00:23:27,259 : INFO : Diretório já existe: /content/COHEBERT_MANUAL


### 5.2.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Copia o arquivo de comparações do google drive para a diretório local
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO"
  
  logging.info("Terminei a cópia!")

2022-07-12 00:23:27,421 : INFO : Terminei a cópia!


Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_PALAVRA_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Descompactação realizada!")

2022-07-12 00:23:27,600 : INFO : Descompactação realizada!


### 3.5.4 Carregamento das lista com os dados do arquivo


In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_comparacao_palavra = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_PALAVRA, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_comparacao_palavra)))

2022-07-12 00:23:27,636 : INFO : TERMINADO ORIGINAIS: 2240.


In [ ]:
lista_comparacao_palavra.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,index_wj,wj,pos_j,cos_mean,euc_mean,man_mean,cos_max,euc_max,man_max
612,18,17,0,0,O,PRON,6,como,ADV,0.774772,26.266504,1275.1833,0.774772,26.266504,1275.1833
181,8,7,0,2,e,CCONJ,7,fila,NOUN,0.670148,29.958317,1460.2004,0.621390,33.349941,1618.4451
1085,25,24,0,5,e,CCONJ,10,nela,PRON,0.542117,38.076847,1879.3391,0.542117,38.076847,1879.3391
286,10,9,0,7,dados,NOUN,9,?,PUNCT,0.589382,35.140377,1721.1606,0.589382,35.140377,1721.1606
1150,26,25,0,5,e,CCONJ,9,elemento,NOUN,0.540975,39.188549,1857.4761,0.540975,39.188549,1857.4761


### 3.5.5 Criando dados indexados

In [ ]:
# Faz uma cópia da lista não indexada
lista_comparacao_palavra_indexado = lista_comparacao_palavra.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_comparacao_palavra_indexado.set_index(["id", "index_sentenca", "index_wi", "index_wj"], inplace=True)
lista_comparacao_palavra_indexado.head()

index_documento    wi  pos_i          wj  \
id index_sentenca index_wi index_wj                                             
1  0              0        1                       0  Como  SCONJ  enfileirar   
                           2                       0  Como  SCONJ   elementos   
                           3                       0  Como  SCONJ          em   
                           4                       0  Como  SCONJ         uma   
                           5                       0  Como  SCONJ        fila   

                                    pos_j  cos_mean   euc_mean   man_mean  \
id index_sentenca index_wi index_wj                                         
1  0              0        1         VERB  0.759586  25.941952  1294.4812   
                           2         NOUN  0.607320  37.124226  1770.7854   
                           3          ADP  0.658418  33.079350  1641.0864   
                           4          DET  0.700411  29.079643  1441.1697   
                           5         NOUN  0.630520  31.865469  1558.0066   

                                      cos_max    euc_max    man_max  
id index_sentenca index_wi index_wj                                  
1  0              0        1         0.673417  31.297091  1554.1917  
                           2         0.607320  37.124226  1770.7854  
                           3         0.658418  33.079350  1641.0864  
                           4         0.700411  29.079643  1441.1697  
                           5         0.578515  35.350060  1716.5850

## 5.3 Carregamento do arquivo de dado comparação palavra e contexto

### 5.3.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_COMPARACAO_CONTEXTO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.3.2 Cria o diretório local para receber os dados


In [ ]:
# Import das bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

2022-07-12 00:23:27,737 : INFO : Diretório já existe: /content/COHEBERT_MANUAL


### 5.3.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Copia o arquivo de comparações do google drive para a diretório local
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO"
  
  logging.info("Terminei a cópia!")

2022-07-12 00:23:27,925 : INFO : Terminei a cópia!


Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Descompactação realizada!")

2022-07-12 00:23:28,090 : INFO : Descompactação realizada!


### 5.3.4 Carregamento das lista com os dados do arquivo


In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_comparacao_contexto = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_CONTEXTO, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_comparacao_contexto))) 

2022-07-12 00:23:28,125 : INFO : TERMINADO ORIGINAIS: 434.


In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
404,38,37,0,6,enfileirar,VERB,0.889269,15.982600,792.9326,0.824856,...,1488.4637,0.707585,29.700611,1433.070923,0.571599,36.291576,1685.0208,0.657645,31.739370,1490.0564
371,35,34,0,13,?,PUNCT,0.771578,24.473789,1217.8474,0.779815,...,1195.6633,0.734280,26.514418,1322.247070,0.665517,30.202028,1473.8259,0.734951,26.385771,1308.3193
387,37,36,0,1,uma,DET,0.840767,20.662043,1027.3184,0.751006,...,1247.7467,0.687070,29.423393,1439.735596,0.690173,29.726408,1401.7122,0.722189,27.461605,1312.7036
205,22,21,0,11,?,PUNCT,0.833500,21.376558,1080.1170,0.800015,...,1159.9232,0.650042,32.918800,1619.813110,0.627764,33.949020,1636.3026,0.709497,28.506548,1418.1788
373,36,35,0,1,são,AUX,0.803046,22.833500,1087.7940,0.757232,...,1184.8553,0.890979,17.435564,834.468384,0.647575,31.281229,1442.3771,0.809497,22.533550,1056.7385


Remove as pontuações

In [ ]:
#lista_comparacao_contexto = lista_comparacao_contexto.query('posi != "PUNCT"')

In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
24,4,3,0,3,em,ADP,0.874367,20.376144,989.66670,0.827974,...,1128.6034,0.769199,26.834612,1314.064453,0.784022,26.464119,1247.2292,0.823296,23.627186,1140.9993
25,4,3,0,4,uma,DET,0.840848,20.481775,994.09705,0.791541,...,1126.5920,0.710721,28.026182,1382.645996,0.735052,27.747614,1295.7932,0.768372,25.028406,1188.0808
155,18,17,0,6,como,ADV,0.840524,21.805305,1098.47130,0.746568,...,1340.7900,0.770056,25.738203,1295.263062,0.632603,33.268932,1629.2722,0.743941,27.018070,1354.7273
234,25,24,0,4,fila,NOUN,0.828565,20.328693,990.87820,0.853684,...,1178.2131,0.668518,30.099642,1466.787842,0.809320,23.541388,1105.3801,0.792520,23.888817,1153.0675
277,28,27,0,11,?,PUNCT,0.826361,21.536108,1082.53470,0.807974,...,1126.6458,0.730823,26.834986,1345.171387,0.650225,31.212452,1530.1620,0.733712,26.466860,1320.0896


In [ ]:
lista_comparacao_contexto.sample(5)

,id,index_documento,index_sentenca,index_wi,wi,pos_i,cos_ctxall_mean,euc_ctxall_mean,man_ctxall_mean,cos_ctxclean_mean,...,man_ctxclean_max,cos_ctxverb_max,euc_ctxverb_max,man_ctxverb_max,cos_ctxnoun_max,euc_ctxnoun_max,man_ctxnoun_max,cos_ctxverbnoun_max,euc_ctxverbnoun_max,man_ctxverbnoun_max
106,13,12,0,2,elementos,NOUN,0.819078,24.533545,1154.30990,0.873726,...,989.80390,NaN,NaN,NaN,0.902469,18.718740,871.16895,0.902469,18.718740,871.16895
246,26,25,0,4,pilha,NOUN,0.756644,26.030293,1165.58470,0.848452,...,952.53253,0.647780,31.523014,1433.916016,0.910936,16.496662,764.73070,0.849135,20.948904,951.64570
228,24,23,0,10,nela,PRON,0.775627,26.351400,1303.82070,0.854318,...,1087.61780,0.566945,37.788605,1829.937988,0.760544,27.346516,1355.01200,0.766785,26.841524,1329.74260
153,18,17,0,4,fila,NOUN,0.828191,20.342186,986.47217,0.860952,...,1156.85860,0.673366,29.684225,1442.485718,0.811382,23.416512,1096.29420,0.795017,23.666025,1147.38720
198,22,21,0,4,pilha,NOUN,0.768251,26.319534,1177.03850,0.860998,...,953.99133,0.506196,40.286327,1834.397217,0.915643,16.683046,777.68774,0.870311,20.238468,933.46640


### 5.3.5 Criando dados indexados

In [ ]:
# Faz uma cópia da lista não indexada
lista_comparacao_contexto_indexado = lista_comparacao_contexto.copy()

# Expecifica o(s) campo(s) indexado(s)
lista_comparacao_contexto_indexado.set_index(["id", "index_sentenca", "index_wi"], inplace=True)
lista_comparacao_contexto_indexado.head()

index_documento          wi  pos_i  \
id index_sentenca index_wi                                       
1  0              0                       0        Como  SCONJ   
                  1                       0  enfileirar   VERB   
                  2                       0   elementos   NOUN   
                  3                       0          em    ADP   
                  4                       0         uma    DET   

                            cos_ctxall_mean  euc_ctxall_mean  man_ctxall_mean  \
id index_sentenca index_wi                                                      
1  0              0                0.828282        21.519108       1075.54500   
                  1                0.957184        10.615781        524.86475   
                  2                0.803867        26.599089       1214.86870   
                  3                0.871055        20.539904        985.52814   
                  4                0.840564        20.068254        958.67480   

                            cos_ctxclean_mean  euc_ctxclean_mean  \
id index_sentenca index_wi                                         
1  0              0                  0.795338          23.453634   
                  1                  0.923698          13.929335   
                  2                  0.868563          22.597618   
                  3                  0.828741          23.203224   
                  4                  0.787430          23.178740   

                            man_ctxclean_mean  cos_ctxverb_mean  ...  \
id index_sentenca index_wi                                       ...   
1  0              0                 1157.0581          0.759586  ...   
                  1                  681.5816          1.000000  ...   
                  2                 1028.9006          0.740469  ...   
                  3                 1111.1733          0.796027  ...   
                  4                 1096.6448          0.744569  ...   

                            man_ctxclean_max  cos_ctxverb_max  \
id index_sentenca index_wi                                      
1  0              0                1157.0581         0.759586   
                  1                1131.7802         0.888775   
                  2                1028.9006         0.740469   
                  3                1111.1733         0.796027   
                  4                1096.6448         0.744569   

                            euc_ctxverb_max  man_ctxverb_max  cos_ctxnoun_max  \
id index_sentenca index_wi                                                      
1  0              0               25.941952      1294.481201         0.680192   
                  1               18.006964       972.863525         0.740781   
                  2               30.121441      1415.033325         0.927326   
                  3               25.312115      1248.880127         0.801384   
                  4               26.130171      1257.120117         0.727427   

                            euc_ctxnoun_max  man_ctxnoun_max  \
id index_sentenca index_wi                                     
1  0              0               29.916761       1453.61890   
                  1               27.314678       1333.59740   
                  2               17.372261        782.10846   
                  3               24.997398       1188.40540   
                  4               27.010202       1257.07130   

                            cos_ctxverbnoun_max  euc_ctxverbnoun_max  \
id index_sentenca index_wi                                             
1  0              0                    0.733323            26.951654   
                  1                    0.819909            22.602350   
                  2                    0.897772            20.224216   
                  3                    0.829802            23.141649   
                  4                    0.760827            24.886763   

                            man_ctxve

## 5.4 Medição



### 5.4.1 Wandb

https://wandb.ai/osmar-braz/MedidaCoerenciaCohebertManual_v1/table?workspace=user-osmar-braz

#### Função de inicialização wandb

In [ ]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project=NOME_BASE_SAIDA, name=NOME_BASE_SAIDA)

    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({'max_seq_len': model_args.max_seq_len})
    wandb.log({'do_lower_case': model_args.do_lower_case})
    wandb.log({'output_hidden_states': model_args.output_hidden_states})
    
    return wandb

### 5.4.2 Função quer realiza a medição dos documentos



#### getMedidasComparacaoPalavra

In [ ]:
import sys

def getMedidasComparacaoPalavra(id_documento, 
                                index_sentenca,
                                index_wi,
                                index_wj,
                                estrategia_medida=0):

  # Pesquisa a medida de comparação das palavras wi e wj
  # pelo id do documento, índice da sentença, índice da palavra wi e índice da palavra wj
  medidas = lista_comparacao_palavra_indexado.loc[id_documento,
                                                  index_sentenca,
                                                  index_wi,
                                                  index_wj]

  if len(medidas) != 0:
    # print(">>>>medidas:",medidas)
    # Seleciona a estratégia de pooling
    pooling = "_mean"
    if estrategia_medida == 1:
      pooling = "_max"

    cos = medidas["cos" + pooling]
    euc = medidas["euc" + pooling]
    man = medidas["man" + pooling]

    return cos, euc, man

  else:
    print("Problemas comparação palavras:", medidas)
    return 0, float(sys.maxsize), float(sys.maxsize)

#### Palavras Adjacentes

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento,
                                                                lista_pos_documento,
                                                                estrategia_medida):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]
      wj = sentenca_token[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = sentenca_pos[ix]
      pos_j = sentenca_pos[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)

      # Recupera as medidas entre wi e wj             
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as k-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord(id_documento,
                                                              documento, 
                                                              lista_sentenca_documento, 
                                                              lista_tokens_documento, 
                                                              lista_pos_documento,
                                                              estrategia_medida):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              
   
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
      
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]
      wj = lista_tokens_nova[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = lista_pos_nova[ix]
      pos_j = lista_pos_nova[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)                    

      # Recupera as medidas entre wi e wj             
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
     
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)
   
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman
    
    del lista_tokens_nova
    del lista_pos_nova    
    
  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes(id_documento,
                                                                    documento, 
                                                                    lista_sentenca_documento, 
                                                                    lista_tokens_documento, 
                                                                    lista_pos_documento,
                                                                    estrategia_medida,
                                                                    classe_saliente=["NOUN","VERB","AUX"]):
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Seleciona somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])    
       
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    #Percorre as palavras do documento    
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]
      wj = lista_tokens_nova[ix+1]
      # Seleciona as POS-Tagging das palavras
      pos_i = lista_pos_nova[ix]
      pos_j = lista_pos_nova[ix+1]

      # print("\nwi:", wi, ix)
      # print("wj:", wj, ix+1)                  

      # Recupera as medidas entre wi e wj                   
      cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
     
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + abs(euc)
      soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
    
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasAdjacentes

Recupera as medidas da coerência das palavras adjacentes de acordo com o filtro de palavras a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaPalavrasAdjacentes(id_documento,
                                          documento, 
                                          lista_sentenca_documento, 
                                          lista_tokens_documento, 
                                          lista_pos_documento, 
                                          estrategia_medida = 0,
                                          filtro_palavra = 0,
                                          classe_saliente=["NOUN","VERB","AUX"]):

  """
    Recupera as medidas da coerência das palavras adjacentes de acordo com o filtro de palavras a ser utilizado nos documentos.
  """    
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasAdjacentesDocumentoTodasPalavras(id_documento, 
                                                                       documento, 
                                                                       lista_sentenca_documento, 
                                                                       lista_tokens_documento, 
                                                                       lista_pos_documento,
                                                                       estrategia_medida)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasAdjacentesDocumentoSemStopWord(id_documento, 
                                                                         documento, 
                                                                         lista_sentenca_documento, 
                                                                         lista_tokens_documento, 
                                                                         lista_pos_documento,
                                                                         estrategia_medida)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasAdjacentesDocumentoPalavrasSalientes(id_documento, 
                                                                                 documento, 
                                                                                 lista_sentenca_documento, 
                                                                                 lista_tokens_documento, 
                                                                                 lista_pos_documento,
                                                                                 estrategia_medida,
                                                                                 classe_saliente=classe_saliente) 

#### Palavras Combinação

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento, 
                                                                lista_pos_documento,
                                                                estrategia_medida):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0

    contaComparacoes = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]      
      pos_i = sentenca_pos[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):

        # Seleciona as palavras do documento  
        wj = sentenca_token[jx]
        pos_j = sentenca_pos[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)

        contaComparacoes = contaComparacoes + 1
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(contaComparacoes)
      MWeuc = float(soma_Weuc)/float(contaComparacoes)
      MWman = float(soma_Wman)/float(contaComparacoes)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord


In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord(id_documento,
                                                              documento, 
                                                              lista_sentenca_documento, 
                                                              lista_tokens_documento, 
                                                              lista_pos_documento,
                                                              estrategia_medida):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              

    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):
      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):
        # Seleciona as palavras do documento  
        wj = lista_tokens_nova[jx]
        pos_j = lista_pos_nova[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova    

  # Calcula a média das medidas para o documento
  Scos = float(soma_Scos)/float(n)
  Seuc = float(soma_Seuc)/float(n)
  Sman = float(soma_Sman)/float(n)
    
  return Scos, Seuc, Sman

##### getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes(id_documento,
                                                                    documento, 
                                                                    lista_sentenca_documento, 
                                                                    lista_tokens_documento, 
                                                                    lista_pos_documento,
                                                                    estrategia_medida,
                                                                    classe_saliente=["NOUN","VERB","AUX"]):
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])
    
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
     # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k-1):
      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # Percorre as palavras da sentença
      for jx in range(ix+1,k):
        # Seleciona as palavras do documento  
        wj = lista_tokens_nova[jx]
        pos_j = lista_pos_nova[jx]

        # print("\nwi:", wi, pos_i)                          
        # print("wj:", wj, pos_j)                    

        # Recupera as medidas entre wi e wj             
        cos, euc, man = getMedidasComparacaoPalavra(id_documento, i, ix, ix+1, estrategia_medida)
  
        # Acumula as medidas do par de palavras
        soma_Wcos = soma_Wcos + abs(cos)
        soma_Weuc = soma_Weuc + abs(euc)
        soma_Wman = soma_Wman + abs(man)

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k-1) != 0:
      MWcos = float(soma_Wcos)/float(k-1)
      MWeuc = float(soma_Weuc)/float(k-1)
      MWman = float(soma_Wman)/float(k-1)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasCombinacao

Recupera as medidas da coerência da combinação de palavras de acordo com o filtro de palavras a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaPalavrasCombinacao(id_documento,
                                          documento, 
                                          lista_sentenca_documento, 
                                          lista_tokens_documento, 
                                          lista_pos_documento,
                                          estrategia_medida,                                
                                          filtro_palavra = 0,
                                          classe_saliente=["NOUN","VERB","AUX"]):
  """
    Recupera as medidas da coerência da combinação de palavras de acordo com o filtro de palavras a ser utilizado nos documentos.
  """  
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasCombinacaoDocumentoTodasPalavras(id_documento, 
                                                                       documento, 
                                                                       lista_sentenca_documento, 
                                                                       lista_tokens_documento, 
                                                                       lista_pos_documento,
                                                                       estrategia_medida)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasCombinacaoDocumentoSemStopWord(id_documento, documento, 
                                                                         lista_sentenca_documento, 
                                                                         lista_tokens_documento, 
                                                                         lista_pos_documento,
                                                                         estrategia_medida)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasCombinacaoDocumentoPalavrasSalientes(id_documento, 
                                                                                 documento, 
                                                                                 lista_sentenca_documento, 
                                                                                 lista_tokens_documento, 
                                                                                 lista_pos_documento,
                                                                                 estrategia_medida,
                                                                                 classe_saliente=classe_saliente)

#### Palavras Global

##### getMedidasComparacaoPalavrasGlobal

In [ ]:
import sys

def getMedidasComparacaoPalavrasGlobal(id_documento, 
                                       index_sentenca, 
                                       index_wi,                                       
                                       estrategia_medida=0,
                                       filtro_palavra=0):
  
  # Pesquisa a medida palavra wi e o contexto
  # pelo id do documento, índice da sentença, índice da palavra wi
  medidas = lista_comparacao_contexto_indexado.loc[id_documento,
                                                   index_sentenca,
                                                   index_wi]                                          

  if len(medidas) != 0:
    # print(">>>>medidas:",medidas)
    # Seleciona a estratégia de pooling
    pooling = "_mean"
    if estrategia_medida == 1:
      pooling = "_max"

    # Seleciona o filtro de palavra
    if filtro_palavra == 0: # Todas as palavras
      cos = medidas["cos_ctxall" + pooling]
      euc = medidas["euc_ctxall" + pooling]
      man = medidas["man_ctxall" + pooling]
    else:
      if filtro_palavra == 1: # Sem as stopwords
        cos = medidas["cos_ctxclean" + pooling]
        euc = medidas["euc_ctxclean" + pooling]
        man = medidas["man_ctxclean" + pooling]
      else:
        if filtro_palavra == 2: # Somente verbos, auxiliar e substantivo
          cos = medidas["cos_ctxverbnoun" + pooling]
          euc = medidas["euc_ctxverbnoun" + pooling]
          man = medidas["man_ctxverbnoun" + pooling]

    return cos, euc, man

  else:
    print("Problemas comparação contexto:", medidas)
    return 0, float(sys.maxsize), float(sys.maxsize)

##### getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras(id_documento,
                                                            documento, 
                                                            lista_sentenca_documento, 
                                                            lista_tokens_documento, 
                                                            lista_pos_documento,
                                                            estrategia_medida,
                                                            filtro_palavra):
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)
  
  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))

    # Quantidade de palavras da sentença
    k = len(sentenca_token)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = sentenca_token[ix]      
      pos_i = sentenca_pos[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i, 
                                                         ix,                                                          
                                                         estrategia_medida,
                                                         filtro_palavra)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + euc
      soma_Wman = soma_Wman + man
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord(id_documento,
                                                          documento, 
                                                          lista_sentenca_documento, 
                                                          lista_tokens_documento, 
                                                          lista_pos_documento,
                                                          estrategia_medida,
                                                          filtro_palavra):
  
  '''
    Percorre as sentenças de um documento para calcular as medidas
  '''
  
  #print("lista_sentenca_documento:", lista_sentenca_documento)
  #print("lista_tokens_documento:", lista_tokens_documento)
  #print("lista_pos_documento:", lista_pos_documento)

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    #print("sentenca:",sentenca)  
    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    #print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    #print("sentenca_pos:",sentenca_pos)
    #print("Quantidade de palavras:",len(sentenca_token))
   
    # Seleciona as palavras e POS-Tagging sem as Stopwords
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, palavra in enumerate(sentenca_token):
        # Se o token estiver não lista de stopwords
        if palavra.lower() not in getStopwords(nlp):
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])              

    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i, 
                                                         ix, 
                                                         estrategia_medida,
                                                         filtro_palavra)
      
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + euc
      soma_Wman = soma_Wman + man
      
    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)
    
    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  Scos = float(soma_Scos)/float(n)
  Seuc = float(soma_Seuc)/float(n)
  Sman = float(soma_Sman)/float(n)
    
  return Scos, Seuc, Sman

##### getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes

In [ ]:
def getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes(id_documento,
                                                                documento, 
                                                                lista_sentenca_documento, 
                                                                lista_tokens_documento, 
                                                                lista_pos_documento,
                                                                estrategia_medida,
                                                                filtro_palavra,
                                                                classe_saliente=["NOUN","VERB","AUX"]):

  # Acumuladores das medidas entre as sentenças    
  soma_Scos = 0
  soma_Seuc = 0
  soma_Sman = 0

  # Quantidade de sentenças no documento
  n = len(lista_sentenca_documento)

  # Percorre as sentenças do documento
  for i, sentenca in enumerate(lista_sentenca_documento):    

    # Carrega as POSTagging da sentença
    sentenca_token = lista_tokens_documento[i]
    # print("sentenca_token:",sentenca_token)
    sentenca_pos = lista_pos_documento[i]
    # print("sentenca_pos:",sentenca_pos)
    # print("Quantidade de palavras:",len(sentenca_token))

    # Somente palavras saliente 
    lista_tokens_nova = []
    lista_pos_nova = []    
    # Percorre as postagging das palavras
    for ix, pos in enumerate(sentenca_pos):
        # Se a postagging da palavra estiver na lista das classes das salientes
        if pos in classe_saliente:
          # Guarda os elementos utilizados
          lista_tokens_nova.append(sentenca_token[ix])
          lista_pos_nova.append(sentenca_pos[ix])
    
    #print("Depois")
    #print("lista_tokens_nova:",lista_tokens_nova)
    #print("POS Tagging:",lista_pos_nova)
    #print("Quantidade de palavras:",len(lista_tokens_nova))

    # Quantidade de palavras na sentença
    k = len(lista_tokens_nova)
  
    # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
     # Acumuladores das medidas entre as palavras  
    soma_Wcos = 0
    soma_Weuc = 0
    soma_Wman = 0
    
    # Seleciona os pares de sentença a serem avaliados
    for ix in range(0,k):

      # Seleciona as palavras do documento  
      wi = lista_tokens_nova[ix]      
      pos_i = lista_pos_nova[ix]
        
      # print("\nwi:", wi, pos_i)      

      # Recupera as medidas entre wi com o contexto
      cos, euc, man = getMedidasComparacaoPalavrasGlobal(id_documento, 
                                                         i, 
                                                         ix, 
                                                         estrategia_medida,
                                                         filtro_palavra)
  
      # Acumula as medidas do par de palavras
      soma_Wcos = soma_Wcos + abs(cos)
      soma_Weuc = soma_Weuc + euc
      soma_Wman = soma_Wman + man

    # Calcula a média das medidas para as m-1 palavras da sentença   
    MWcos = 0
    MWeuc = 0
    MWman = 0

    # Se existe palavras
    if float(k) != 0:
      MWcos = float(soma_Wcos)/float(k)
      MWeuc = float(soma_Weuc)/float(k)
      MWman = float(soma_Wman)/float(k)

    # Acumula a média das medidas das palavras da sentença
    soma_Scos = soma_Scos + MWcos
    soma_Seuc = soma_Seuc + MWeuc
    soma_Sman = soma_Sman + MWman

    del lista_tokens_nova
    del lista_pos_nova

  # Calcula a média das medidas para o documento
  MScos = float(soma_Scos)/float(n)
  MSeuc = float(soma_Seuc)/float(n)
  MSman = float(soma_Sman)/float(n)
  
  return MScos, MSeuc, MSman

##### getMedidasCoerenciaPalavrasGlobal

Recupera as medidas da palavra com o contexto global de acordo com o filtro de palavras a ser utilizado nos documentos.



In [ ]:
def getMedidasCoerenciaPalavrasGlobal(id_documento, 
                                      documento, 
                                      lista_sentenca_documento, 
                                      lista_tokens_documento, 
                                      lista_pos_documento,
                                      estrategia_medida,                             
                                      filtro_palavra = 0,
                                      classe_saliente=["NOUN","VERB","AUX"]):
  
  """
    Recupera as medidas da palavra com o contexto global de acordo com o filtro de palavras a ser utilizado nos documentos.
  """
  
  if filtro_palavra == 0: # Todas as palavras
    return getMedidasCoerenciaPalavrasGlobalDocumentoTodasPalavras(id_documento, 
                                                                   documento, 
                                                                   lista_sentenca_documento, 
                                                                   lista_tokens_documento, 
                                                                   lista_pos_documento,
                                                                   estrategia_medida,
                                                                   filtro_palavra)
  else:
    if filtro_palavra == 1: # Sem stopwords
        return getMedidasCoerenciaPalavrasGlobalDocumentoSemStopWord(id_documento, documento, 
                                                                     lista_sentenca_documento, 
                                                                     lista_tokens_documento, 
                                                                     lista_pos_documento,
                                                                     estrategia_medida,
                                                                     filtro_palavra)
    else: 
        if filtro_palavra == 2: # Somente verbos(e auxiliares) substantivos          
          return getMedidasCoerenciaPalavrasGlobalDocumentoPalavrasSalientes(id_documento, 
                                                                             documento, 
                                                                             lista_sentenca_documento, 
                                                                             lista_tokens_documento, 
                                                                             lista_pos_documento,
                                                                             estrategia_medida,
                                                                             filtro_palavra,
                                                                             classe_saliente=classe_saliente)        
  

#### getMedidasCoerenciaDocumento

Recupera as medidas de coerência de acordo com a equação a ser utilizado nos documentos.

In [ ]:
def getMedidasCoerenciaDocumento(id_documento,                                  
                                 documento, 
                                 lista_sentenca_documento, 
                                 lista_tokens_documento, 
                                 lista_pos_documento,
                                 equacao_medida = 0,
                                 estrategia_medida = 0,
                                 filtro_palavra = 0,
                                 classe_saliente=["NOUN","VERB","AUX"]):
  
  """
    Recupera as medidas de coerência de acordo com a equação a ser utilizado nos documentos.
  """
  
  if equacao_medida == 0: # Compara palavras adjacentes
    # print("Calculando para palavra adjacentes")
    return getMedidasCoerenciaPalavrasAdjacentes(id_documento, 
                                                 documento, 
                                                 lista_sentenca_documento, 
                                                 lista_tokens_documento, 
                                                 lista_pos_documento, 
                                                 estrategia_medida,
                                                 filtro_palavra,
                                                 classe_saliente=classe_saliente)
  else:
    if equacao_medida == 1: # Compara todas as palavras
        # print("Calculando para todas as palavras")
        return getMedidasCoerenciaPalavrasCombinacao(id_documento, 
                                                     documento, 
                                                     lista_sentenca_documento, 
                                                     lista_tokens_documento, 
                                                     lista_pos_documento, 
                                                     estrategia_medida,
                                                     filtro_palavra,
                                                     classe_saliente=classe_saliente)
    else:         
        if equacao_medida == 2: # Somente verbos(e auxiliares) substantivos          
          # print("Calculando para o contexto")  
          return getMedidasCoerenciaPalavrasGlobal(id_documento, 
                                                   documento, 
                                                   lista_sentenca_documento, 
                                                   lista_tokens_documento, 
                                                   lista_pos_documento,
                                                   estrategia_medida,
                                                   filtro_palavra, 
                                                   classe_saliente=classe_saliente)
  

### 5.4.3 Função que realiza a medição de todos os documentos

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

def calculaMedidasDocumentos(lista_documentos_agrupados,                              
                             wandb, 
                             equacao_medida, 
                             estrategia_medida,
                             filtro_palavra):
    
  '''
    Percorre os documentos para calcular as medidas das sentenças
  '''

  logging.info("Processando {} pares de documentos originais e perturbados.".format(len(lista_documentos_agrupados)))

  # Contadores de ocorrência de coerência
  conta_Ccos = 0
  conta_Ceuc = 0
  conta_Cman = 0
  conta = 0

  # Lista para o salvamento das medidas
  lista_medidas_documentos_salvar = []

  # Barra de progresso dos documentos
  lista_documentos_agrupados_bar = tqdm_notebook(lista_documentos_agrupados.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_agrupados))

  # Percorre os documentos do conjunto de dados
  for i, linha_documento in lista_documentos_agrupados_bar:  
    # if i < 10:     

      # Conta o número de pares de documentos
      conta = conta + 1

      #print("linha_documento:",linha_documento)
      # Recupera o id do documento Original
      id_documento_original = linha_documento[0]   

      # print("id_documento_original:",id_documento_original)    
      lista_sentenca_documento_original = linha_documento[1]     
      #print("lista_sentenca_documento_original:",lista_sentenca_documento_original)
      #print("len(lista_sentenca_documento_original):",len(lista_sentenca_documento_original))
      # Recupera o documento Original
      documento_original = linha_documento[2]    
      #print("documento_original:",documento_original) 
      # Recupera os tokens do documento original
      lista_tokens_documento_original = linha_documento[3]
      #print("lista_tokens_documento_original:",lista_tokens_documento_original)
      #print("len(lista_tokens_documento_original):",len(lista_tokens_documento_original))
      # Recupera o postagging do documento original
      lista_pos_documento_original = linha_documento[4]
      #print("lista_pos_documento_original:",lista_pos_documento_original)
      #print("len(lista_pos_documento_original):",len(lista_pos_documento_original))

      Ccos_orig, Ceuc_orig, Cman_orig = getMedidasCoerenciaDocumento(id_documento_original,
                                                                     documento_original, 
                                                                     lista_sentenca_documento_original,
                                                                     lista_tokens_documento_original,
                                                                     lista_pos_documento_original,
                                                                     equacao_medida=equacao_medida,
                                                                     estrategia_medida=estrategia_medida,
                                                                     filtro_palavra=filtro_palavra)
               
      # Recupera o id do documento Perturbado
      id_documento_perturbado = linha_documento[5]     
      # print("id_documento_perturbado:",id_documento_perturbado)    
      lista_sentenca_documento_perturbado = linha_documento[6]     
      #print("lista_sentenca_documento_perturbado:",lista_sentenca_documento_perturbado)
      #print("len(lista_sentenca_documento_perturbado):",len(lista_sentenca_documento_perturbado))
      # Recupera o documento Perturbado
      documento_perturbado = linha_documento[7]
      #print("documento_perturbado:",documento_perturbado) 
      # Recupera os tokens do documento perturbado
      lista_tokens_documento_perturbado = linha_documento[8]
      #print("lista_tokens_documento_perturbado:",lista_tokens_documento_perturbado)
      #print("len(lista_tokens_documento_perturbado):",len(lista_tokens_documento_perturbado))
      # Recupera o postagging do documento original
      lista_pos_documento_perturbado = linha_documento[9]
      #print("lista_pos_documento_perturbado:",lista_pos_documento_perturbado)
      #print("len(lista_pos_documento_perturbado):",len(lista_pos_documento_perturbado))
      
      Ccos_pert, Ceuc_pert, Cman_pert =  getMedidasCoerenciaDocumento(id_documento_perturbado,
                                                                      documento_perturbado, 
                                                                      lista_sentenca_documento_perturbado, 
                                                                      lista_tokens_documento_perturbado, 
                                                                      lista_pos_documento_perturbado,
                                                                      equacao_medida=equacao_medida,
                                                                      estrategia_medida=estrategia_medida,
                                                                      filtro_palavra=filtro_palavra)
    
      # Verifica a medida de coerência Ccos(similaridade do cosseno) das sentenças do documento original com as sentenças do documento permutado.
      # Quanto maior o valor de Ccos mais as orações do documentos são coerentes
      if Ccos_orig >= Ccos_pert:
          conta_Ccos = conta_Ccos + 1

      # Verifica a medida de incoerência Seuc(distância euclidiana) das sentenças do documento original com as sentenças do documento permutado.
      # Quanto menor o valor de Ceuc mais as orações do documentos são coerentes
      if Ceuc_orig <= Ceuc_pert:
          conta_Ceuc = conta_Ceuc + 1

      # Verifica a medida de incoerência Sman(distância de manhattan) das sentenças do documento original com as sentenças do documento permutado.
      # Quanto menor o valor de Cman mais as orações do documentos são coerentes
      if Cman_orig <= Cman_pert:
          conta_Cman = conta_Cman + 1        

      # Guarda as medidas em uma lista para salvar em arquivo
      # Guarda as medidas dos documentos originais
      lista_medidas_documentos_salvar.append([id_documento_original, 
                                              Ccos_orig,  
                                              Ceuc_orig,  
                                              Cman_orig])
      # Guarda as medidas dos documentos perturbados
      lista_medidas_documentos_salvar.append([id_documento_perturbado, 
                                              Ccos_pert, 
                                              Ceuc_pert, 
                                              Cman_pert])
      
  logging.info("Total de Pares : {}.".format(str(conta)))

  if model_args.use_wandb:
       wandb.log({'pares_doc': conta})

  logging.info("Pares Corretos Ccos: {}.".format(str(conta_Ccos)))
  acuracia_Ccos = float(conta_Ccos)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Ccos*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_ccos': acuracia_Ccos})

  logging.info("Pares Corretos Ceuc: {}.".format(str(conta_Ceuc)))
  acuracia_Ceuc = float(conta_Ceuc)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Ceuc*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_ceuc': acuracia_Ceuc})  

  logging.info("Pares Corretos Cman: {}.".format(str(conta_Cman)))
  acuracia_Cman = float(conta_Cman)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_Cman*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_cman': acuracia_Cman})  

  logging.info("TERMINADO!")

  del lista_documentos_agrupados_bar
 
  return lista_medidas_documentos_salvar, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman

### 5.4.4 Salvando os resultados

#### Salvando o resultado da medição

In [ ]:
def salvaResultadoMedicao(lista_medidas_documentos_salvar):

  if model_args.salvar_medicao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    EQUACAO = '_adj' # adjacentes
    if model_args.equacao_medida == 1:
        EQUACAO = '_com' # combinação de todas as palavras
    else:
      if model_args.equacao_medida == 2: 
         EQUACAO = '_ctx' # contexto

    ESTRATEGIA = '_mean'
    if model_args.estrategia_medida == 1:
      ESTRATEGIA = '_max'

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords     
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs'  # Somente verbos(e auxiliares) e substantivos

    AJUSTADO = '_pretreinado'
    if model_args.usar_mcl_ajustado == True:
        AJUSTADO = '_ajustado'

    # Contatena os parâmetros que forma o nome do arquivo medição    
    NOME_ARQUIVO_MEDICAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + EQUACAO + ESTRATEGIA + FILTRO_PALAVRA + getNomeModeloBERT(model_args) + getTamanhoBERT(model_args) + AJUSTADO 
    
    # Diretório do drive do arquivo
    DIRETORIO_MEDICAO_DRIVE = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao/"

    # Diretório local para salvar o arquivo
    DIRETORIO_MEDICAO_LOCAL = DIRETORIO_LOCAL + "Medicao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_DRIVE):  
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_DRIVE)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_DRIVE))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_DRIVE))

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_LOCAL):  
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_LOCAL)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_LOCAL))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_LOCAL))

    # Caminho completo do arquivo compactado no drive
    NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO = DIRETORIO_MEDICAO_DRIVE + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO:", NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO)

    # Caminho completo do arquivo compactado no local
    NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO:", NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO)

    # Caminho completo do arquivo no local
    NOME_ARQUIVO_MEDICAO_LOCAL = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".csv"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL:", NOME_ARQUIVO_MEDICAO_LOCAL)

    # Gera todo o conteúdo a ser salvo no arquivo
    novo_conteudo = ''        
    for resultado in lista_medidas_documentos_salvar:      
      novo_conteudo = novo_conteudo + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' + str(resultado[0]) + ';'  + str(resultado[1]) + ';'  + str(resultado[2]) + ';'  + str(resultado[3]) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO):
      # Copia arquivo da medição compactado do google drive para o drive local
      !cp "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO"  
      # Descompacta arquivo da medição compactado no drive local
      !unzip -o -j -q "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" -d "$DIRETORIO_MEDICAO_LOCAL"

      logging.info("Atualizando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_COMPLETO,'w')
      # Escreve o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL" "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"  

    else:
      logging.info("Criando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'w')
      arquivo.writelines('data;arquivo;ccos;ceuc;cman\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL" "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"

In [ ]:
def salvaResultadoMedicao1(lista_medidas_documentos_salvar):

  if model_args.salvar_medicao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    EQUACAO = '_adj' # adjacentes
    if model_args.equacao_medida == 1:
        EQUACAO = '_com' # combinação de todas as palavras
    else:
      if model_args.equacao_medida == 2: 
         EQUACAO = '_ctx' # contexto

    ESTRATEGIA = '_mean'
    if model_args.estrategia_medida == 1:
      ESTRATEGIA = '_max'

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords     
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs'  # Somente verbos(e auxiliares) e substantivos

    AJUSTADO = '_pretreinado'
    if model_args.usar_mcl_ajustado == True:
        AJUSTADO = '_ajustado'

    # Nome arquivo resultado    
    NOME_ARQUIVO_MEDICAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + EQUACAO + ESTRATEGIA + FILTRO_PALAVRA + getNomeModeloBERT(model_args) + getTamanhoBERT(model_args) + AJUSTADO 
    
    # Diretório para salvar o arquivo.    
    DIRETORIO_MEDICAO = DIRETORIO_DRIVE + "validacao_medicao_palavra/Medicao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_MEDICAO_COMPLETO = DIRETORIO_MEDICAO + NOME_ARQUIVO_MEDICAO + '.csv'

    # Gera todo o conteúdo a ser salvo no arquivo
    novo_conteudo = ''        
    for resultado in lista_medidas_documentos_salvar:      
      novo_conteudo = novo_conteudo + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' + str(resultado[0]) + ';'  + str(resultado[1]) + ';'  + str(resultado[2]) + ';'  + str(resultado[3]) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_MEDICAO_COMPLETO):
      logging.info("Atualizando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_MEDICAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_COMPLETO,'w')
      # Escreve o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()
    else:
      logging.info("Criando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_COMPLETO,'w')
      arquivo.writelines('data;arquivo;ccos;ceuc;cman\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

#### Salvando o resultado da avaliação

In [ ]:
def salvaResultadoAvaliacao(tempo_total_processamento, 
                            conta, 
                            acuracia_Ccos, 
                            conta_Ccos, 
                            acuracia_Ceuc, 
                            conta_Ceuc, 
                            acuracia_Cman, 
                            conta_Cman):

  if model_args.salvar_avaliacao:

    # Import das bibliotecas.
    import os

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    EQUACAO = '_adj' # adjacentes
    if model_args.equacao_medida == 1:
        EQUACAO = '_com' # combinação de todas as palavras
    else:
      if model_args.equacao_medida == 2: 
         EQUACAO = '_ctx' # contexto

    ESTRATEGIA = '_mean'
    if model_args.estrategia_medida == 1:
      ESTRATEGIA = '_max'

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords     
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs' # Somente verbos(e auxiliares) e substantivos   

    AJUSTADO = '_pretreinado'
    if model_args.usar_mcl_ajustado == True:
        AJUSTADO = '_ajustado'

    # Nome arquivo resultado    
    NOME_ARQUIVO_AVALIACAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + EQUACAO + ESTRATEGIA + FILTRO_PALAVRA + getNomeModeloBERT(model_args) + getTamanhoBERT(model_args) + AJUSTADO

    # Diretório para salvar o arquivo de resultado.
    DIRETORIO_AVALIACAO = DIRETORIO_DRIVE + "validacao_medicao_palavra/Avaliacao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_AVALIACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_AVALIACAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_AVALIACAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_AVALIACAO))
    
    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + '.csv'
    
    # Conteúdo a ser adicionado.
    novo_conteudo = NOME_ARQUIVO_AVALIACAO + ';' + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' + tempo_total_processamento + ';'  + str(conta) + ';'  + str(acuracia_Ccos) + ';' + str(conta_Ccos) + ';' + str(acuracia_Ceuc) + ';' + str(conta_Ceuc) + ';' + str(acuracia_Cman) + ';' + str(conta_Cman) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      logging.info("Atualizando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()
    else:
      logging.info("Criando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')      
      arquivo.writelines('arquivo;data;tempo;conta;ccos;contaccos;ceuc;contaceuc;cman;contacman\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

### 5.4.5 Função de cálculo das medidas dos documentos

In [ ]:
# Import das bibliotecas.
import time
import datetime
import gc

def procedimentoCalculaMedida(equacao_medida, 
                              estrategia_medida, 
                              filtro_palavra):

  # Seta o parâmetro da equação
  model_args.equacao_medida = equacao_medida
  # Seta o parâmetro da estratégia
  model_args.estrategia_medida = estrategia_medida
  # Seta o parâmetro do fitro
  model_args.filtro_palavra = filtro_palavra

  logging.info("Processamento a equação {}, com estratégia {} e com filtro palavra {}.".format(EQUACAO_MEDIDA_STR[equacao_medida], ESTRATEGIA_MEDIDA_STR[estrategia_medida], FILTRO_PALAVRAS_STR[filtro_palavra]))
      
  # Marca o tempo de início do processamento
  tempoInicioTeste = time.time()
  logging.info("Tempo início processamento: {:} (h:mm:ss).".format(formataTempo(tempoInicioTeste)))

  # Inicializa o wandb para registro
  wandb = inicializacaoWandb()
  
  # Calcula as medidas dos documentos  
  resultado_medida, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman = calculaMedidasDocumentos(lista_documentos_agrupados, wandb, equacao_medida, estrategia_medida, filtro_palavra)
  # print("resultado_medida:", resultado_medida)

  # Pega o tempo atual menos o tempo do início do processamento.
  tempoFinalTeste = time.time()
  tempo_total_processamento = formataTempo(tempoFinalTeste - tempoInicioTeste)

  # Salva o resultado da classificação
  salvaResultadoMedicao(resultado_medida)
  
  # Salva o resultado da avaliação
  salvaResultadoAvaliacao(tempo_total_processamento, conta, acuracia_Ccos, conta_Ccos, acuracia_Ceuc, conta_Ceuc, acuracia_Cman, conta_Cman)

  logging.info("  Tempo processamento: {:} (h:mm:ss).\n".format(tempo_total_processamento))

  # Finaliza o wandb
  if model_args.use_wandb:
     wandb.finish()  

  # Apaga as variáveis
  del resultado_medida  
  del wandb

  # Chama o coletor de lixo para esvaziar a memória
  gc.collect()  

### 5.4.6 Executa o procedimento para todos os parâmetros

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

# Barra de progresso equação(0 - Adjacentes(CAW), 1 - Combinação(CCW), 2 - Globa (CGW))
equacao_medida_bar = tqdm_notebook(enumerate(EQUACAO_MEDIDA), desc=f'Equações', unit=f'equação', total=len(EQUACAO_MEDIDA))

# Percorre todos as equações a serem avaliados
for equacao_medida_i, equacao_medida in equacao_medida_bar:

  # Barra de progresso estratégias(0 - MEAN e 1 - MAX para as palavras formadas por mais de um token do BERT)
  estrategia_medida_bar = tqdm_notebook(enumerate(ESTRATEGIA_MEDIDA), desc=f'Estratégias', unit=f'estratégia', total=len(ESTRATEGIA_MEDIDA))

  # Percorre todos as estratégias a serem avaliados
  for estrategia_medida_i, estrategia_medida in estrategia_medida_bar:

    # Barra de progresso filtro (0 - ALL, 1 - CLEAN, 2 - VERBNOUN)
    filtro_palavra_bar = tqdm_notebook(enumerate(FILTRO_PALAVRAS), desc=f'Filtro palavras', unit=f'filtro', total=len(FILTRO_PALAVRAS))

    # Percorre todos formas de filtro de palavras a serem avaliados
    for filtro_palavra_i, filtro_palavra in filtro_palavra_bar:
      
      # Passa os parâmetros para o procedimento cálculo das medidas
      procedimentoCalculaMedida(equacao_medida, estrategia_medida, filtro_palavra)

Equações:   0%|          | 0/3 [00:00<?, ?equação/s]

Estratégias:   0%|          | 0/2 [00:00<?, ?estratégia/s]

Filtro palavras:   0%|          | 0/3 [00:00<?, ?filtro/s]

2022-07-12 00:23:29,768 : INFO : Processamento a equação ADJACENTES, com estratégia MEAN e com filtro palavra TODAS_AS_PALAVRAS.
2022-07-12 00:23:29,772 : INFO : Tempo início processamento: 19185 days, 0:23:30 (h:mm:ss).


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


2022-07-12 00:26:22,221 : INFO : Processando 20 pares de documentos originais e perturbados.


Documentos:   0%|          | 0/20 [00:00<?, ? documento/s]

2022-07-12 00:26:22,443 : INFO : Total de Pares : 20.
2022-07-12 00:26:22,447 : INFO : Pares Corretos Ccos: 11.
2022-07-12 00:26:22,453 : INFO : Acurácia: 55.00000000000001.
2022-07-12 00:26:22,457 : INFO : Pares Corretos Ceuc: 10.
2022-07-12 00:26:22,460 : INFO : Acurácia: 50.0.
2022-07-12 00:26:22,463 : INFO : Pares Corretos Cman: 11.
2022-07-12 00:26:22,467 : INFO : Acurácia: 55.00000000000001.
2022-07-12 00:26:22,472 : INFO : TERMINADO!
2022-07-12 00:26:22,476 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra/Medicao/.
2022-07-12 00:26:22,480 : INFO : Criando arquivo medição: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra/Medicao/MedidaCoerenciaCohebertManual_v1_P_1_K_1_ctx_max_svs_BERTimbau_large_pretreinado.csv.
2022-07-12 00:26:22,497 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_medicao_palavra/Avaliacao/.
2022-07-12 00:2

acuracia_ccos,▁
acuracia_ceuc,▁
acuracia_cman,▁
do_lower_case,▁
max_seq_len,▁
output_hidden_states,▁
pares_doc,▁
acuracia_ccos,0.55
acuracia_ceuc,0.5
acuracia_cman,0.55
do_lower_case,False


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print('')
print('  Tempo processamento:  {:} (h:mm:ss)'.format(tempo_total_processamento))


  Tempo processamento:  0:03:47 (h:mm:ss)
